In [1]:
import pandas as pd
import numpy as np
import dask.dataframe as dd
#this is for imputing data properly
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer 

#this is for regularization
from sklearn.linear_model import Lasso
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score

#imputation files 
import imputation
from util import get_bad_columns

In [2]:
bad_col = get_bad_columns('../data/Data-Codebook.pdf')
bad_col.remove('WB')
bad_col

['METHDES',
 'DIETPILS',
 'RITMPHEN',
 'STMCARD',
 'CYLERT',
 'DEXED',
 'DETAMP',
 'DIDREX',
 'ESKAT',
 'IONAMIN',
 'MAZANOR',
 'OBLA',
 'PLEGINE',
 'PRELUDIN',
 'SANOREX',
 'TENUATE',
 'STMNOLST',
 'STIMNEWA',
 'STIMNEWB',
 'STIMNEWC',
 'STIMNEWD',
 'STIMNEWE',
 'STMEVER',
 'STIMAGE',
 'STIMYFU',
 'STIMMFU',
 'STIMREC',
 'STMYRTOT',
 'STTOTFG',
 'STFQFLG',
 'STBSTWAY',
 'STDAYPYR',
 'STDAYPMO',
 'STDAYPWK',
 'METHAGE',
 'METHYFU',
 'METHMFU',
 'METHREC',
 'MTHYRTOT',
 'MTTOTFG',
 'MTFQFLG',
 'MTBSTWAY',
 'MTDAYPYR',
 'MTDAYPMO',
 'MTDAYPWK',
 'COCEVER',
 'COCAGE',
 'COCYFU',
 'COCMFU',
 'COCREC',
 'COCYRTOT',
 'CCTOTFG',
 'CCFQFLG',
 'CCBSTWAY',
 'CCDAYPYR',
 'CCDAYPMO',
 'CCDAYPWK',
 'COCUS30A',
 'CC30EST',
 'CRKEVER',
 'CRKAGE',
 'CRKYFU',
 'CRKMFU',
 'CRAKREC',
 'CRKYRTOT',
 'CRTOTFG',
 'CRFQFLG',
 'CRBSTWAY',
 'CRDAYPYR',
 'CRDAYPMO',
 'CRDAYPWK',
 'CRKUS30A',
 'CR30EST',
 'HEREVER',
 'HERAGE',
 'HERYFU',
 'HERMFU',
 'HERREC',
 'HERYRTOT',
 'HRTOTFG',
 'HRFQFLG',
 'HRBSTWAY',
 'HR

In [3]:
df = pd.read_table("../data/35509-0001-Data.tsv")
df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,99,99,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,99,99,16,9999,99,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,99,99,991,9991,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,991,9991,91,91,91,...,9,99,9,9,1,1,2,3104.06,30051,2


In [4]:
#drop all columns that already imply drug use
cleaned_df = df.drop(bad_col, axis = 1)
cleaned_df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,99,99,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.40,30017,1
1,2,88530883,1,99,99,14,9999,99,2,93,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,99,99,14,9999,99,1,2,...,1,99,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,99,99,16,9999,99,4,93,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,99,99,14,9999,99,4,93,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,99,99,991,9991,91,91,91,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,99,99,16,9999,99,4,93,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,99,99,991,9991,91,91,91,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,991,9991,91,91,91,...,9,99,9,9,1,1,2,3104.06,30051,2


In [5]:
cleaned_df = cleaned_df.applymap(imputation.check_missing)
cleaned_df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,<NA>,<NA>,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,<NA>,<NA>,16,<NA>,<NA>,4,<NA>,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,<NA>,<NA>,14,<NA>,<NA>,4,<NA>,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,<NA>,<NA>,16,<NA>,<NA>,4,<NA>,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,...,9,<NA>,9,9,1,1,2,3104.06,30051,2


In [6]:
cleaned_df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGYFU,CIGMFU,CIGREC,CIG30USE,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,<NA>,<NA>,19,2012,7,1,1,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,<NA>,<NA>,14,<NA>,<NA>,2,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,<NA>,<NA>,14,<NA>,<NA>,1,2,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,<NA>,<NA>,16,<NA>,<NA>,4,<NA>,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,<NA>,<NA>,14,<NA>,<NA>,4,<NA>,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,<NA>,<NA>,16,<NA>,<NA>,4,<NA>,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,...,9,<NA>,9,9,1,1,2,3104.06,30051,2


In [7]:
good_columns = cleaned_df.columns[cleaned_df.isnull().sum()/cleaned_df.shape[0] < 0.95]
#keep only columns with 0-95% of values being null
#this leaves us with 1800ish columns
good_columns

Index(['CASEID', 'QUESTID2', 'CIGEVER', 'CIGOFRSM', 'CIGWILYR', 'CIGTRY',
       'CIGREC', 'CIG30USE', 'CIG30AV', 'CIG30BR2',
       ...
       'II2EMSTY', 'EMPSTAT4', 'IIEMPST4', 'II2EMST4', 'PDEN00', 'COUTYP2',
       'MAIIN002', 'ANALWT_C', 'VESTR', 'VEREP'],
      dtype='object', length=1831)

In [8]:
cleaned_df = cleaned_df.loc[:,good_columns]
cleaned_df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGREC,CIG30USE,CIG30AV,CIG30BR2,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,<NA>,<NA>,19,1,1,2,<NA>,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,<NA>,<NA>,14,2,<NA>,<NA>,<NA>,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,<NA>,<NA>,14,1,2,1,112,...,1,<NA>,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,<NA>,<NA>,16,4,<NA>,<NA>,<NA>,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,<NA>,<NA>,14,4,<NA>,<NA>,<NA>,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,<NA>,<NA>,16,4,<NA>,<NA>,<NA>,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,<NA>,<NA>,<NA>,<NA>,<NA>,...,9,<NA>,9,9,1,1,2,3104.06,30051,2


In [9]:
cleaned_df = cleaned_df.apply(imputation.impute_na_with_random)
"""
this imputes the data probabilistically, however it could lead to poor
correlation for certain variables with high NAs values
""" 
cleaned_df

,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGREC,CIG30USE,CIG30AV,CIG30BR2,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
0,1,48694667,1,4,4,19,1,1,2,112,...,1,4,1,1,2,2,2,4398.4,30017,1
1,2,88530883,1,4,4,14,2,30,2,104,...,1,4,1,1,1,1,2,1419.19,30052,2
2,3,33251077,1,4,4,14,1,2,1,112,...,1,2,9,9,3,3,2,14052.62,30028,1
3,4,37814127,1,4,4,16,4,1,3,104,...,1,4,1,1,1,1,2,10848.18,30055,2
4,5,18762590,1,4,4,14,4,30,4,104,...,1,1,1,1,2,2,2,5651.73,30013,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55155,55156,13675473,2,4,4,13,1,10,4,112,...,1,2,1,1,3,3,2,679.36,30003,1
55156,55157,49609908,1,4,4,16,4,30,6,118,...,1,1,1,1,2,2,2,2296.28,30057,2
55157,55158,81795924,2,4,4,17,1,15,4,118,...,1,4,1,1,3,3,2,17180.64,30026,1
55158,55159,17198338,2,4,4,30,1,30,4,119,...,9,1,9,9,1,1,2,3104.06,30051,2


In [10]:
X, y = cleaned_df.drop("HLTINDRG", axis = 1), cleaned_df["HLTINDRG"]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [12]:
#scale the data appropriately 
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [13]:
#after playing around with alpha, low alphas brought the best results
#for r2 and mse
lasso= Lasso(alpha = 0.05)
lasso.fit(X_train, y_train)

Lasso(alpha=0.05)

In [14]:
y_pred = lasso.predict(X_test)

In [15]:
#sees how good the model is 
mse = mean_squared_error(y_test, y_pred)
mse

0.14696363577489438

In [16]:
r2 = r2_score(y_test, y_pred)
r2

0.09807853989947568

In [17]:
feature_importance = lasso.coef_

In [18]:
features_name = []
importance_lst = []
for feature, importance in zip(list(X.columns), feature_importance):
    features_name.append(feature)
    importance_lst.append(importance)
features_name = pd.Series(features_name)
importance_ser = pd.Series(importance_lst)
importance_df = pd.DataFrame([features_name, importance_ser])
importance_df

,0,1,2,3,4,5,6,7,8,9,...,1820,1821,1822,1823,1824,1825,1826,1827,1828,1829
0,CASEID,QUESTID2,CIGEVER,CIGOFRSM,CIGWILYR,CIGTRY,CIGREC,CIG30USE,CIG30AV,CIG30BR2,...,II2EMSTY,EMPSTAT4,IIEMPST4,II2EMST4,PDEN00,COUTYP2,MAIIN002,ANALWT_C,VESTR,VEREP
1,0.0,-0.0,-0.0,-0.0,0.0,0.0,0.0,-0.0,-0.0,-0.0,...,-0.0,0.0,-0.0,-0.0,0.0,0.0,-0.0,-0.0,-0.0,-0.0


In [19]:
importance_df = importance_df.T
importance_df = importance_df.rename(columns = {0: 'feature', 1:'importance'})
importance_df

,feature,importance
0,CASEID,0.0
1,QUESTID2,-0.0
2,CIGEVER,-0.0
3,CIGOFRSM,-0.0
4,CIGWILYR,0.0
...,...,...
1825,COUTYP2,0.0
1826,MAIIN002,-0.0
1827,ANALWT_C,-0.0
1828,VESTR,-0.0


In [20]:
importance_df.sort_values(by = "importance", ascending = False)
#this only caught 3 features as non zero
# IEMFLAG, MJOFLAG, MRJFLAG which is only marjuiana(was hoping for more)

,feature,importance
1731,HLTINALC,0.060461
1732,HLTINMNT,0.036584
0,CASEID,0.0
1229,YETCGJOB,0.0
1227,YESCHIMP,-0.0
...,...,...
606,RKHERREG,0.0
605,RKTRYHER,0.0
604,RKLSDREG,0.0
603,RKTRYLSD,0.0
